<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

***Создание таблица по ТОП сайтам с просмотрами более 1000 просмотров***

In [1]:
import pandas as pd
import pyarrow as pa
import pyarrow.feather as feather
import pyarrow.compute  as pc
import polars as pl
import numpy as np

In [2]:
read_arrow = feather.read_table('dataset_full.feather')

In [3]:
%%time
data_agg = read_arrow.select(['user_id', 'url_host', 'request_cnt']).\
    group_by(['user_id', 'url_host']).aggregate([('request_cnt', "sum")])

CPU times: total: 12.8 s
Wall time: 13.5 s


In [4]:
%%time
top_url = read_arrow.select(['url_host', 'request_cnt']).\
    group_by(['url_host']).aggregate([('request_cnt', "sum")]).filter(pc.field('request_cnt_sum') >= 1000)

CPU times: total: 4.02 s
Wall time: 4.23 s


In [5]:
list_url = list(pl.from_arrow(top_url['url_host']))

In [6]:
print(f'{len(list_url)} сайтов с просмотрами более 1000')

3436 сайтов с просмотрами более 1000


In [7]:
top_url_data = data_agg.filter(pc.field('url_host').isin(list_url))

In [8]:
data = pl.from_arrow(top_url_data)

In [9]:
data.head()

request_cnt_sum,user_id,url_host
i64,i32,cat
12,45098,"""ad.adriver.ru"""
491,45098,"""apple.com"""
174,45098,"""avatars.mds.ya..."
305,45098,"""googleads.g.do..."
446,45098,"""i.ytimg.com"""


In [10]:
data = data.with_columns(pl.col('request_cnt_sum').cast(pl.Int32))

In [11]:
data.head(1)

request_cnt_sum,user_id,url_host
i32,i32,cat
12,45098,"""ad.adriver.ru"""


In [12]:
del top_url, data_agg, top_url_data, read_arrow

In [13]:
data = data.pivot(index='user_id', columns='url_host', values='request_cnt_sum')

In [14]:
data.head(1)

user_id,ad.adriver.ru,apple.com,avatars.mds.yandex.net,googleads.g.doubleclick.net,i.ytimg.com,icloud.com,m.avito.ru,relap.io,sun9-5.userapi.com,ads.adfox.ru,banners.adfox.ru,mail.yandex.ru,syndication.realsrv.com,ad.mail.ru,vk.com,ssp.otm-r.com,ads.betweendigital.com,s.optnx.com,yastatic.net,dsp.rtb.mts.ru,tpc.googlesyndication.com,festima.ru,sun9-85.userapi.com,s.viiadr.com,login.mts.ru,sun9-77.userapi.com,aliexpress.ru,t.me,yandex.ru,rtb.com.ru,onlinetours.ru,mamba.ru,exchange.buzzoola.com,forms.yandex.ru,sun9-80.userapi.com,rt.bongacams14.com,...,hawk.live,radiopiterfm.ru,dh-online.ru,seasonhit.tk,sudoku.com,merojax.me,gotovim-online.ru,w3m.huawei.com,zvukiknig.net,transphoto.org,online.anidub.com,forum.omskmama.ru,xiaomi.com,huawei.fotostrana.ru,news-show.online,wordle.belousov.one,part-kom.ru,rusanime.ru,v-5ka.ru,hmn.ru,x5.advantum.ru,jaomix.ru,ladys-room.ru,info.flyredwings.com,wap1.3101010.ru,ranobes.com,doctorhouse.tv,v01.ru,d3.taxinonstop.ru,avalon.antorgps.com,tatarlove.ru,dengi-white.ru,a-kniga.online,mixmuz.ru,loveplanet.cc,online.cheboko.com,murdrum.net
i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,...,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32
45098,12,491,174,305,446,188,82,5,4,54,3,29,7,47,46,2,16,5,60,5,2,1,7,1,17,8,5,1,61,3,1,1,7,1,7,2,...,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null


In [15]:
data = data.fill_null(0)

In [17]:
data.head(1)

user_id,ad.adriver.ru,apple.com,avatars.mds.yandex.net,googleads.g.doubleclick.net,i.ytimg.com,icloud.com,m.avito.ru,relap.io,sun9-5.userapi.com,ads.adfox.ru,banners.adfox.ru,mail.yandex.ru,syndication.realsrv.com,ad.mail.ru,vk.com,ssp.otm-r.com,ads.betweendigital.com,s.optnx.com,yastatic.net,dsp.rtb.mts.ru,tpc.googlesyndication.com,festima.ru,sun9-85.userapi.com,s.viiadr.com,login.mts.ru,sun9-77.userapi.com,aliexpress.ru,t.me,yandex.ru,rtb.com.ru,onlinetours.ru,mamba.ru,exchange.buzzoola.com,forms.yandex.ru,sun9-80.userapi.com,rt.bongacams14.com,...,hawk.live,radiopiterfm.ru,dh-online.ru,seasonhit.tk,sudoku.com,merojax.me,gotovim-online.ru,w3m.huawei.com,zvukiknig.net,transphoto.org,online.anidub.com,forum.omskmama.ru,xiaomi.com,huawei.fotostrana.ru,news-show.online,wordle.belousov.one,part-kom.ru,rusanime.ru,v-5ka.ru,hmn.ru,x5.advantum.ru,jaomix.ru,ladys-room.ru,info.flyredwings.com,wap1.3101010.ru,ranobes.com,doctorhouse.tv,v01.ru,d3.taxinonstop.ru,avalon.antorgps.com,tatarlove.ru,dengi-white.ru,a-kniga.online,mixmuz.ru,loveplanet.cc,online.cheboko.com,murdrum.net
i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,...,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32
45098,12,491,174,305,446,188,82,5,4,54,3,29,7,47,46,2,16,5,60,5,2,1,7,1,17,8,5,1,61,3,1,1,7,1,7,2,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [28]:
print(len(data['user_id'].unique()), 'уникальных пользователей для 3436 сайтов')

415307 уникальных пользователей для 3436 сайтов


In [29]:
data.write_parquet('data_url.parquet') 